# ThreadPoolExecutor线程池
        Python中已经有了threading模块，为什么还需要线程池呢，线程池又是什么东西呢？在介绍线程同步的信号量机制的时候，举得例子是爬虫的例子，需要控制同时爬取的线程数，例子中创建了20个线程，而同时只允许3个线程在运行，但是20个线程都需要创建和销毁，线程的创建是需要消耗系统资源的，有没有更好的方案呢？其实只需要三个线程就行了，每个线程各分配一个任务，剩下的任务排队等待，当某个线程完成了任务的时候，排队任务就可以安排给这个线程继续执行。

        这就是线程池的思想（当然没这么简单），但是自己编写线程池很难写的比较完美，还需要考虑复杂情况下的线程同步，很容易发生死锁。从Python3.2开始，标准库为我们提供了concurrent.futures模块，它提供了ThreadPoolExecutor和ProcessPoolExecutor两个类，实现了对threading和multiprocessing的进一步抽象（这里主要关注线程池），不仅可以帮我们自动调度线程，还可以做到：
        1.主线程可以获取某一个线程（或者任务的）的状态，以及返回值。
        2.当一个线程完成的时候，主线程能够立即知道。
        3.让多线程和多进程的编码接口一致。
- 参数
    - ThreadPoolExecutor构造实例的时候，传入max_workers参数来设置线程池中最多能同时运行的线程数目。
    - 使用submit函数来提交线程需要执行的任务（函数名和参数）到线程池中，并返回该任务的句柄（类似于文件、画图），注意submit()不是阻塞的，而是立即返回。
    - 通过submit函数返回的任务句柄，能够使用done()方法判断该任务是否结束。上面的例子可以看出，由于任务有2s的延时，在task1提交后立刻判断，task1还未完成，而在延时4s之后判断，task1就完成了。
    - 使用cancel()方法可以取消提交的任务，如果任务已经在线程池中运行了，就取消不了。这个例子中，线程池的大小设置为2，任务已经在运行了，所以取消失败。如果改变线程池的大小为1，那么先提交的是task1，task2还在排队等候，这是时候就可以成功取消。
    - 使用result()方法可以获取任务的返回值。查看内部代码，发现这个方法是阻塞的。

- 详情见
            链接：https://www.jianshu.com/p/b9b3d66aa0be

# ProcessPoolExecutor进程池

# concurrent.futures
- 为异步执行可调用对象提供了高层接口
- 该模块提供了两大类型:
    - Executor类：用来管理工作池,包含ThreadPoolExecutor和ProcessPoolExecutor两个类，现了对threading和multiprocessing的进一步抽象
    - Future类：管理工作计算出来的结果，通常不直接操作future对象，因为有丰富的接口

# concurrent.futures.Executor类
- 提供了一系列方法，用于异步执行调用，不能直接调用，只能通过子类化出来的具体类来使用：两大子类为ThreadPoolExecutor和ProcessPoolExecutor
- 定义的方法有submit,map,shutdown
    - sumbit(fn, *args, **kwargs):
        - 安排可调用对象fn以fn(*args, **kwargs)的形式执行，并返回Future对象来表示他们的执行
    - map(func, *iterables, timeout=None, chunksize=1):
        - 类似Python内置函数map(func, *iteaables)，但是有两点不同
            - 1、立即获取iterables而不会惰性获取
            - 2、异步执行func，并支持多次并发调用
        - 返回一个迭代器，如果超时，如果迭代不出可用结果后，迭代器抛出concurrent.futures.TimeoutError异常
        - 如果func调用抛出了异常，那么该异常会在迭代器获取值得时候抛出
        - 当使用ProcessPoolExcutor时，这个方法会把iterables划分成多个块，作为独立的任务提交到进程池。这些块的带下可用通过给定的chunksize指定一个正整数，对于很长的iterables，使用较大的chunksize可以显著提高性能。这个参数只对进ProcessPoolExcutor起作用
        - 不管并发的任务的执行次序如何，map总是基于输入顺序来返回值
    - shutdown(wait=True):
        - 告诉执行器executor在当前所有等待的future对象运行完毕后，应该释放执行器用到的所有资源
        - shutdown之后不可再submit或者map
        - 如果wait为True，则此方法会在所有等待的future都执行完毕，并且属于执行器executor的资源都释放后才返回
        - 如果wait为Flase，本方法会立即返回，属于执行器的资源会在所有等待的future执行完毕后释放
        - 无论wait为何值，整个Python程序在等待的future执行完毕之前不会退出
        - 可以通过with语句来避免显式调用本方法，即默认wait为True
        - 执行器类实现了上下文协议，可以用作上下去管理器。它能并发执行任务，等待它们全部完成。当上下文管理器退出时，自动调用 shutdown() 方法。
    
    
# concurrent.futures.Future
- 封装了可调用对象的异步执行。
- Future实例通过Executor.submit()创建，除非用于测试，否则不应该直接手动创建
- 实例方法：with concurrent.futures.ProcessPoolExecutor() as executor
    - future.cancel() 尝试取消调用，如果该调用正在执行中，无法取消，本方法返回 False，其他情况下调用会被取消，并返回 True。
    - running() 如果调用正在执行，无法被取消，则返回 True。
    - done() 如果调用成功被取消或者已经执行完毕，返回 True。
    - result(timeout=None) 返回调用的返回值。
        - 如果调用还没有完成，则最多等待 timeout 秒。如果 timeout 秒之后还没有完成，抛出 concurrent.futures.TimeoutError异常。timeout 可以为整数或者浮点数。如果不指定或者为 None，则不限制等待时间。如果 future 在完成之前被取消了，会抛出 CancelledError 异常。
        - 如果调用时就抛出异常，这个方法会抛出同样的异常。
    - exception(timeout=None)返回被调用抛出的异常。
        - timeout方法同上

    - add_done_callback(callback)
        - 为 future 附加可调用对象callback。当 future 运行完毕或者被取消时，它会被用作 callbak 的唯一参数，并调用 callback。
        - 可调用对象按照添加顺序依次调用，并且总是在添加时所处进程的一个线程内调用它。如果该可调用对象抛出了属于 Exception 子类的异常，它会被记录并忽略。如果它抛出了属于 BaseException 子类的异常，该行为未定义。
        - 如果 future 已经完成或者已经取消，fn 会被立即调用。 
        
- 模块方法（类方法）
    - concurrent.futures.wait(fs, timeout=None, return_when=ALL_COMPLETED)
        - 等待 Future 实例完成，这些实例可能由多个不同的执行器实例创建，通过 fs 指定这些 Future 实例。返回具名元组，该元组有两个元素，每个元素都是一个集合。第一个元素名叫 done，该集合包括已完成的 futures；第二个元素名叫 pending，该集合包括未完成的 futures。
        - return_when 指明函数何时应该返回。它必须是下列常量之一：
                FIRST_COMPLETED：函数在任意一个 future 完成或者被取消时返回。
                FIRST_EXCEPTION：函数在任意一个 future 因为异常而结束时返回。如果没有 future 抛出异常，它等价于 ALL_COMPLETED。
                ALL_COMPLETED：当所有 future 完成或者被取消时函数才会返回。
                
     - concurrent.futures.as_completed(fs, timeout=None)
         - 当通过 fs 指定的 Future 实例全部执行完毕或者被取消后，返回这些 Future 实例组成的迭代器。fs 中的 Future 实例可以被不同的执行器创建。任何在 as_completed() 调用之前就已经完成的 Future 实例会被最先生成。

- Futuer内部方法：用于单元测试和实现自定义执行器的方法
     - set_running_or_notify_cancel()
     - set_result(result)
     - set_exception(exception)

# coroutine.futures.Future与asyncio.Future之间的区别
        asyncio中的Future类是模仿concurrent.futures.Future类而设计的，关于concurrent.futures.Future，可以查阅相关的文档。它们之间的主要区别是：

        （1）asyncio.Future对象是awaitable的，但是concurrent.futures.Future对象是不能够awaitable的；

        （2）asyncio.Future.result()和asyncio.Future.exception()是不接受关键字参数timeout的；

        （3）当Future没有完成的时候，asyncio.Future.result()和asyncio.Future.exception()将会触发一个InvalidStateError异常；

        （4）使用asyncio.Future.add_done_callback()注册的回调函数不会立即执行，它可以使用loop.call_soon代替；

        （5）asyncio里面的Future和concurrent.futures.wait()以及concurrent.futures.as_completed()是不兼容的。


# ThreadPoolExecutor线程执行器
- 通过线程池来执行异步调用，它管理一个组工作线程，当工作线程有富余的时候，给他们传递任务。当属于一个Future对象的可调用对象等待另一个Future的返回时，会发生死锁。
- concurrent.futures.ThreadPoolExecutor(max_workers=None, thread_name_prefix='', initializer=None, initargs=())
        这个 Executor 子类最多用 max_workers 个线程来异步执行调用。

        initializer 是一个可选的可调用对象，会在每个 worker 线程启动之前调用。
        initargs 是传递给 initializer 的参数元组。
        如果 initializer 抛出了异常，那么当前所有等待的任务都会抛出 BrokenThreadPool 异常，继续提交 submit 任务也会抛出此异常。

        3.5 的变化：如果 max_worker 没有指定或者为 None，则默认为本机处理器数量乘以 5。

        3.6 新特性：添加了 thread_name_prefix 参数，可以控制由线程池创建的工作线程名称，便于调试。

        3.7 的变化：添加了 initializer 和 initargs 参数。

In [3]:
# 死锁举例
import time
from concurrent.futures import ThreadPoolExecutor

def waiting_a():
    time.sleep(2)
    print(b.result())
    print("waiting_a工作结束")
    
def waiting_b():
    time.sleep(1)
    print(a.result())
    print("waiting_b工作结束")
    
excutor = ThreadPoolExecutor(max_workers=2)
a = excutor.submit(waiting_b)
b = excutor.submit(waiting_a)
print(a, b)

<Future at 0x25a8cd5aa58 state=running> <Future at 0x25a8cb6afd0 state=running>


# ProcessPoolExecutor 进程池执行器
- ProcessPoolExecutor 进程池执行器类是 Executor 执行器类的子类，使用进程池来异步执行调用。
- ProcessPoolExecutor 使用了 multiprocessing 模块，这允许它可以规避 Global Interpreter Lock，但是也意味着只能执行和返回可序列化的（picklable）对象。

        __main__ 模块必须被 worker 子进程导入，这意味着 ProcessPoolExecutor 在交互解释器中无法工作。

        在已经被提交到 ProcessPoolExecutor 中的可调用对象内使用 Executor 或者 Future 方法会导致死锁。

        concurrent.futures.ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=())
        这个 Executor 子类最多用 max_workers 个进程来异步执行调用。
        如果不指定 max_workers 或者为 None，它默认为本机的处理器数量。
        如果 max_workers 小于等于 0，会抛出 ValueError 异常。
        mp_context 是多进程上下文（multiprocessing context）或者 None，它会被用来启动 workers。如果不指定 mp_context 或者为 None，会使用默认的多进程上下文环境。

        initializer 是一个可选的可调用对象，会在每个 worker 进程启动之前调用。
        initargs 是传递给 initializer 的参数元组。
        如果 initializer 抛出了异常，那么当前所有等待的任务都会抛出 BrokenProcessPool 异常，继续提交 submit 任务也会抛出此异常。

        3.3 版本的变化：任意一个工作进程突然中止时，会抛出 BrokenProcessPool 异常。之前版本中，行为是未定义的，而且对于执行器或者它的 future 对象的操作通常会无响应或者死锁。

        3.7 版本的变化：加入了 mp_context 参数，允许用户控制由进程池创建的工作进程的 start_method 方法。该版本还加入了 initializer 和 initargs 参数。



In [6]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()
    # 在交互命令中不能使用进程池

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.